In [ ]:
%load_ext autoreload
%autoreload 2

import albumentations as A
import cv2
import random

from tqdm import tqdm
from pathlib import Path

from drone_detection.augment_tools import read_bboxes, save_bboxes

In [ ]:
# NOTE: Paths from another project!

root_path = Path().resolve()

# Origin data
data_path = root_path / "origin_wout_augment"

images_dir = data_path / "images"
labels_dir = data_path / "labels"

# Augmented data
data_augment_path = root_path / "origin_with_augment"
aug_images_dir = data_augment_path / "images"
aug_labels_dir = data_augment_path / "labels"

In [ ]:
first_augmentations = [
    A.HorizontalFlip(p=0.9),
    A.RandomRotate90(p=0.9),
    A.Rotate(p=0.9),
    A.Affine(p=0.9),
]

second_augmentations = [
    A.RandomBrightnessContrast(p=0.9),
    A.GaussNoise(p=0.9),
    A.HueSaturationValue(p=0.9),
    A.ToGray(p=0.4),
    A.RandomRain(blur_value=2, p=0.9),
    A.RGBShift(p=0.9),
]

In [ ]:
augment_iter = 4
lst_images_dir = list(images_dir.glob("*.jpg"))

for aug_iter in range(augment_iter):
    for image_filename in tqdm(lst_images_dir, desc="Images", total=len(lst_images_dir)):
        label_name = str(image_filename.stem) + ".txt"
        label_path = labels_dir / label_name

        image = cv2.imread(image_filename)
        bboxes, class_labels = read_bboxes(label_path)

        rnd_1_aug = random.choice(first_augmentations)
        rnd_2_aug, rnd_3_aug = random.sample(second_augmentations, 2)

        transform = A.Compose(
            transforms=[rnd_1_aug, rnd_2_aug, rnd_3_aug],
            bbox_params=A.BboxParams(format="yolo", label_fields=["class_labels"]),
        )

        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        transformed_image = transformed["image"]
        transformed_bboxes = transformed["bboxes"]
        transformed_cls_labels = transformed["class_labels"]
                
        aug_img_filename = f"{image_filename.stem}_aug_{aug_iter}_iter.jpg"
        aug_img_path = aug_images_dir / aug_img_filename
        
        aug_label_filename = f"{image_filename.stem}_aug_{aug_iter}_iter.txt"
        aug_label_path = aug_labels_dir / aug_label_filename

        cv2.imwrite(aug_img_path, transformed_image)
        save_bboxes(aug_label_path, transformed_bboxes, transformed_cls_labels)